# Notebook 1: RealSense RGB + Depth Preview
This notebook captures live RGB and depth frames from the Intel RealSense D455 camera. It allows you to preview the streams and save snapshots by pressing 's'. Press 'q' to quit.

In [ ]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os


In [ ]:
# Initialize RealSense pipeline
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

# Start the camera
pipeline.start(config)

print("Press 's' to save a frame, 'q' to quit.")


In [ ]:
try:
    while True:
        # Wait for a coherent pair of frames
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()

        if not depth_frame or not color_frame:
            continue

        # Convert to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())

        # Visualize depth
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(depth_image, alpha=0.03),
            cv2.COLORMAP_JET
        )

        # Combine images
        combined = np.hstack((color_image, depth_colormap))
        cv2.imshow("RealSense RGB + Depth", combined)

        key = cv2.waitKey(1)
        if key == ord('s'):
            os.makedirs("snapshots", exist_ok=True)
            idx = len(os.listdir("snapshots")) // 2
            cv2.imwrite(f"snapshots/rgb_{idx}.png", color_image)
            np.save(f"snapshots/depth_{idx}.npy", depth_image)
            print(f"Saved rgb_{idx}.png and depth_{idx}.npy")
        elif key == ord('q'):
            break

finally:
    pipeline.stop()
    cv2.destroyAllWindows()
